Text Classification Using FastText

Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification
We have a dataset of ecommerce item description. Total 4 categories,

Household
Electronics
Clothing and Accessories
Books
The task at hand is to classify a product into one of the above 4 categories based on the product description

In [3]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [4]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [6]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_5364\2912538940.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


In [7]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When you train a fasttext model, it expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [8]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [10]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


Pre-procesing

Remove punctuation
Remove extra space
Make the entire sentence lower case

In [11]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [12]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [13]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


Train Test Split

In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [15]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [16]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

Train the model and evaluate performance

In [17]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(10083, 0.9668749380144798, 0.9668749380144798)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

Now let's do prediction for few product descriptions

In [18]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99730307]))

In [19]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [20]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [21]:
model.get_nearest_neighbors("painting")

[(0.9989529848098755, 'yozo'),
 (0.9989243149757385, 'aerates'),
 (0.9989243149757385, 'durable\xa0'),
 (0.9989243149757385, 'price\xa0'),
 (0.9989243149757385, 'free\xa0'),
 (0.9989243149757385, 'dripping\xa0'),
 (0.9989243149757385, 'aeration'),
 (0.9989243149757385, 'multiples'),
 (0.9989243149757385, 'wines\xa0'),
 (0.9989243149757385, 'aerators')]

In [22]:
model.get_nearest_neighbors("sony")

[(0.9991161227226257, 'lenovotm'),
 (0.999116063117981, '9210'),
 (0.999116063117981, '80tda00hin'),
 (0.9991071224212646, 'campatible'),
 (0.9991071224212646, 'accesory'),
 (0.999096155166626, '5pclfk67'),
 (0.9990363121032715, '5421'),
 (0.9990363121032715, '5437'),
 (0.9990363121032715, '3421'),
 (0.9990363121032715, '40whr')]